In [1]:
from glob import glob
from tqdm import tqdm
import json
import numpy as np
import os
from transformers import AutoTokenizer

In [2]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
files = sorted(glob('output/*.json'), key = lambda x: int(x.split('-')[1].replace('.json', '')))
len(files)

36539

In [4]:
!du -hs output-audio

280G	output-audio


In [5]:
# import IPython.display as ipd
# ipd.Audio('output-audio/3-5833-0.mp3')

In [6]:
# for f in tqdm(files):
#     with open(f) as fopen:
#         data = json.load(fopen)

In [7]:
import mp
import copy

def loop(files):
    files, _ = files
    results = []
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
        f_split = os.path.split(f)[-1].replace('.json', '')
        for i in range(len(data)):
            
            audio_filename = os.path.join('output-audio', f'{f_split}-{i}.mp3')
            if not os.path.exists(audio_filename):
                continue
            
            data[i]['audio_filename'] = audio_filename
            data[i]['filename'] = f
            data[i]['i'] = i
            a = np.array(tokenizer.encode(data[i]['predict_ms'], add_special_tokens = False))
            a = a[a != 50257]
            data[i]['predict_ms'] = tokenizer.decode(a.tolist() + [50257])
            a = np.array(tokenizer.encode(data[i]['predict_en'], add_special_tokens = False))
            a = a[a != 50257]
            data[i]['predict_en'] = tokenizer.decode(a.tolist() + [50257])
            
            results.append(data[i])
    return results

In [8]:
results = mp.multiprocessing(files, loop, cores = 20)

100%|██████████| 1826/1826 [03:03<00:00,  9.93it/s]


In [9]:
len(results)

1899976

In [10]:
results[-1]

{'predict_ms': '<|startoftranscript|><|ms|><|transcribe|> Kamu tahu bahawa kami berkongsi? Banyak kemasan yang kamu bawa? Saya mahu berkongsi. Tidak, saya perlu menghapusnya. Okey, area ini jangan menyentuh. Boleh Jermaine? Tidak, saya hanya bergurau. Saya telah bekerja dengan Jermaine untuk beberapa pembentangan sebelum ini. Seperti di belakang, untuk rehat dan saya telah melihat dia bersedia dari dirinya yang biasa ke dirinya yang glamour.<|endoftext|>',
 'predict_en': "<|startoftranscript|><|en|><|transcribe|> You know that we're sharing room. How much makeup did you bring ah? Do I want to share? No! I need to demark it. Okay this area don't touch okay Jermaine? No I'm just kidding. I worked with Jermaine for quite a couple of shows before like backstage for rehearsals and I've seen her get ready from like her normal self to her glamorous self.<|endoftext|>",
 'score_ms': 5.8125,
 'score_en': 8.6875,
 'audio_filename': 'output-audio/3-9172-51.mp3',
 'filename': 'output/3-9172.json',

In [11]:
import IPython.display as ipd
ipd.Audio(results[-1]['audio_filename'])

In [12]:
with open('pseudolabel.jsonl', 'w') as fopen:
    for r in tqdm(results):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 1899976/1899976 [00:20<00:00, 92486.93it/s] 


In [13]:
!ls -lh pseudolabel.jsonl

-rwxrwxrwx 1 ubuntu ubuntu 1.7G Dec 28 03:12 pseudolabel.jsonl


In [14]:
from huggingface_hub import HfApi
api = HfApi()

In [ ]:
api.upload_file(
    path_or_fileobj='pseudolabel.jsonl',
    path_in_repo='pseudolabel.jsonl',
    repo_id='mesolitica/pseudolabel-malaysian-youtube-whisper-large-v3',
    repo_type='dataset',
)

pseudolabel.jsonl:   0%|          | 0.00/1.75G [00:00<?, ?B/s]